Start by importing `Magma`.

In [1]:
import magma as m

Import a FullAdder from `Mantle`. 

In [2]:
import os
os.environ['MANTLE_TARGET'] = 'ice40'
from mantle import FullAdder

print('{} [{}]'.format(str(FullAdder), str(FullAdder.interface)))

import mantle lattice ice40
import mantle lattice mantle40
FullAdder ["I0", In(Bit), "I1", In(Bit), "CIN", In(Bit), "O", Out(Bit), "COUT", Out(Bit)]


Now let's build a n-bit adder from the FullAdder. 

![2-Bit Adder](logisim/adder.png)

In [3]:
def adder(A, B, Cin):
    n = len(A)
    C = []
    Cout = Cin
    for i in range(n):
        fulladder = FullAdder()
        m.wire(A[i], fulladder.I0)
        m.wire(B[i], fulladder.I1)
        m.wire(Cout, fulladder.CIN)
        C.append(fulladder.O)
        Cout = fulladder.COUT
    return m.uint(C), Cout 

Initialize the IceStick headers. `J1` will be used as inputs; `J3` will be used as outputs.

In [4]:
N = 2

from loam.boards.icestick import IceStick

icestick = IceStick()
for i in range(N):
    icestick.J1[i].input().on()
    icestick.J1[i+N].input().on()
    icestick.J3[i].output().on()

In [5]:
main = icestick.DefineMain()

s, c = adder(main.J1[0:N], main.J1[N:N+N], 0)
m.wire( s, main.J3 )

m.EndDefine()

In [6]:
m.compile('build/adder', main)

compiling FullAdder
compiling main


In [7]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif adder.blif' adder.v
arachne-pnr -q -d 1k -o adder.txt -p adder.pcf adder.blif 
icepack adder.txt adder.bin

/Users/hanrahan/git/magmathon/notebooks/build


In [8]:
%%bash 
# iceprog build/adder.bin

Show verilog. FullAdder has been mapped to ice40 primitives.

In [9]:
with open("build/adder.v", "r") as adder_verilog:
    print(adder_verilog.read())

module FullAdder (input  I0, input  I1, input  CIN, output  O, output  COUT);
wire  inst0_O;
wire  inst1_CO;
SB_LUT4 #(.LUT_INIT(16'h9696)) inst0 (.I0(I0), .I1(I1), .I2(CIN), .I3(1'b0), .O(inst0_O));
SB_CARRY inst1 (.I0(I0), .I1(I1), .CI(CIN), .CO(inst1_CO));
assign O = inst0_O;
assign COUT = inst1_CO;
endmodule

module main (input [3:0] J1, output [1:0] J3);
wire  inst0_O;
wire  inst0_COUT;
wire  inst1_O;
wire  inst1_COUT;
FullAdder inst0 (.I0(J1[0]), .I1(J1[2]), .CIN(1'b0), .O(inst0_O), .COUT(inst0_COUT));
FullAdder inst1 (.I0(J1[1]), .I1(J1[3]), .CIN(inst0_COUT), .O(inst1_O), .COUT(inst1_COUT));
assign J3 = {inst1_O,inst0_O};
endmodule




Show physical constraints file. FPGA pins are connected to headers J1 and J2.

In [10]:
with open("build/adder.pcf", "r") as adder_verilog:
    print(adder_verilog.read())

set_io J1[0] 112
set_io J1[1] 113
set_io J1[2] 114
set_io J1[3] 115
set_io J3[1] 61
set_io J3[0] 62

